In [1]:
import sys
import logging
import requests

import pandas as pd

from tqdm import tqdm_notebook as tqdm
from concurrent.futures import ThreadPoolExecutor

In [2]:
logging.basicConfig()

pd.options.display.max_columns = None

In [3]:
BASEURL = 'https://query1.finance.yahoo.com'

In [4]:
notnone = lambda x: x is not None

In [5]:
stocks = pd.read_csv('../datasets/stocks.csv.gz', index_col=0)

stocks.head()

,ticker,name,exchange,category name,country
0,OEDV,"Osage Exploration and Development, Inc.",PNK,NaN,USA
1,AAPL,Apple Inc.,NMS,Electronic Equipment,USA
2,BAC,Bank of America Corporation,NYQ,Money Center Banks,USA
3,AMZN,"Amazon.com, Inc.",NMS,Catalog & Mail Order Houses,USA
4,T,AT&T Inc.,NYQ,Telecom Services - Domestic,USA


In [6]:
stocks.describe(include='all')

,ticker,name,exchange,category name,country
count,106328,96676,95092,20728,95091
unique,106328,64859,67,214,41
top,BGWTATO.BO,FINANC/TERMO,PNK,Industrial Metals & Minerals,USA
freq,1,439,14602,1292,22169


In [7]:
tickers = stocks['ticker']

tickers.head()

0    OEDV
1    AAPL
2     BAC
3    AMZN
4       T
Name: ticker, dtype: object

In [8]:
def download_info(ticker):
    url = f"{BASEURL}/v7/finance/quote?symbols={ticker}"
    
    try:
        res = requests.get(url=url).json()
        result = res['quoteResponse']['result'][0]
        
        return result
    except Exception as e:
        # fail silently
        return None

In [10]:
with ThreadPoolExecutor() as executor:
    gen = executor.map(download_info, tickers)
    infos = pd.DataFrame(filter(notnone, tqdm(gen, total=len(tickers))))

print(infos.shape)

infos.head()


(81929, 75)


,language,region,quoteType,quoteSourceName,regularMarketTime,marketState,exchangeDataDelayedBy,sourceInterval,exchangeTimezoneName,exchangeTimezoneShortName,gmtOffSetMilliseconds,esgPopulated,tradeable,triggerable,priceHint,fullExchangeName,market,exchange,symbol,currency,regularMarketPrice,regularMarketChange,regularMarketOpen,regularMarketDayHigh,regularMarketDayLow,regularMarketVolume,dividendDate,earningsTimestamp,earningsTimestampStart,earningsTimestampEnd,trailingAnnualDividendRate,trailingPE,trailingAnnualDividendYield,epsTrailingTwelveMonths,epsForward,sharesOutstanding,bookValue,fiftyDayAverage,fiftyDayAverageChange,fiftyDayAverageChangePercent,twoHundredDayAverage,twoHundredDayAverageChange,twoHundredDayAverageChangePercent,marketCap,forwardPE,priceToBook,shortName,regularMarketPreviousClose,bid,ask,bidSize,askSize,messageBoardId,longName,financialCurrency,regularMarketChangePercent,regularMarketDayRange,averageDailyVolume3Month,averageDailyVolume10Day,fiftyTwoWeekLowChange,fiftyTwoWeekLowChangePercent,fiftyTwoWeekRange,fiftyTwoWeekHighChange,fiftyTwoWeekHighChangePercent,fiftyTwoWeekLow,fiftyTwoWeekHigh,ytdReturn,trailingThreeMonthReturns,trailingThreeMonthNavReturns,openInterest,underlyingSymbol,contractSymbol,expireIsoDate,expireDate,headSymbolAsString
0,en-US,US,MUTUALFUND,Delayed Quote,1.561760e+09,REGULAR,0.0,15.0,America/New_York,EDT,-14400000.0,False,False,False,2.0,YHD,us_market,YHD,OEDV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,en-US,US,EQUITY,Nasdaq Real Time Price,1.566829e+09,REGULAR,0.0,15.0,America/New_York,EDT,-14400000.0,False,True,True,2.0,NasdaqGS,us_market,NMS,AAPL,USD,205.750,3.110001,205.86,207.1900,205.53,7381238.0,1.565827e+09,1.564531e+09,1.572448e+09,1.572887e+09,2.96,17.470493,0.014607,11.777,12.80,4.601080e+09,21.286,205.056560,0.693436,0.003382,193.434420,12.315582,0.063668,9.298212e+11,16.074219,9.665978,Apple Inc.,202.64,206.47,206.51,10.0,10.0,finmb_24937,Apple Inc.,USD,1.53474,205.53 - 207.19,26427811.0,28254233.0,63.750000,0.448944,142.0 - 233.47,-27.720001,-0.118730,142.00,233.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,en-US,US,EQUITY,Nasdaq Real Time Price,1.566829e+09,REGULAR,0.0,15.0,America/New_York,EDT,-14400000.0,False,True,True,2.0,NYSE,us_market,NYQ,BAC,USD,26.525,0.055000,26.70,26.7400,26.50,7086157.0,1.569542e+09,1.571223e+09,1.515677e+09,1.516023e+09,0.60,9.439502,0.022667,2.810,3.04,9.508200e+09,26.408,28.741714,-2.216715,-0.077125,28.784782,-2.259783,-0.078506,2.469027e+11,8.725329,1.004430,Bank of America Corporation,26.47,26.59,26.60,361.0,14.0,finmb_19049,Bank of America Corporation,USD,0.207784,26.5 - 26.74,52258963.0,57081883.0,3.865000,0.170565,22.66 - 31.49,-4.965000,-0.157669,22.66,31.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,en-US,US,EQUITY,Nasdaq Real Time Price,1.566829e+09,REGULAR,0.0,15.0,America/New_York,EDT,-14400000.0,False,True,True,2.0,NasdaqGS,us_market,NMS,AMZN,USD,1756.000,6.380005,1766.91,1769.8206,1751.40,849731.0,NaN,1.564099e+09,1.571843e+09,1.572278e+09,NaN,72.850975,NaN,24.104,33.22,4.920530e+08,107.411,1888.932000,-132.932000,-0.070374,1824.243400,-68.243410,-0.037409,8.686159e+11,52.859722,16.348417,"Amazon.com, Inc.",1749.62,1757.77,1757.42,8.0,14.0,finmb_18749,"Amazon.com, Inc.",USD,0.364651,1751.4 - 1769.8206,3624549.0,2953316.0,449.000000,0.343535,1307.0 - 2050.5,-294.500000,-0.143623,1307.00,2050.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,en-US,US,EQUITY,Nasdaq Real Time Price,1.566829e+09,REGULAR,0.0,15.0,America/New_York,EDT,-14400000.0,False,True,True,2.0,NYSE,us_market,NYQ,T,USD,34.895,0.075001,34.97,35.0600,34.86,2932219.0,1.564618e+09,1.571848e+09,1.571762e+09,1.572280e+09,2.03,14.717419,0.058300,2.371,3.62,6.495230e+09,25.223,34.012287,0.882713,0.025953,32.070724,2.824276,0.088064,2.549778e+11,9.639504,1.383460,AT&T Inc.,34.82,34.95,34.96,18.0,40.0,finmb

In [11]:
infos.describe(include='all')

,language,region,quoteType,quoteSourceName,regularMarketTime,marketState,exchangeDataDelayedBy,sourceInterval,exchangeTimezoneName,exchangeTimezoneShortName,gmtOffSetMilliseconds,esgPopulated,tradeable,triggerable,priceHint,fullExchangeName,market,exchange,symbol,currency,regularMarketPrice,regularMarketChange,regularMarketOpen,regularMarketDayHigh,regularMarketDayLow,regularMarketVolume,dividendDate,earningsTimestamp,earningsTimestampStart,earningsTimestampEnd,trailingAnnualDividendRate,trailingPE,trailingAnnualDividendYield,epsTrailingTwelveMonths,epsForward,sharesOutstanding,bookValue,fiftyDayAverage,fiftyDayAverageChange,fiftyDayAverageChangePercent,twoHundredDayAverage,twoHundredDayAverageChange,twoHundredDayAverageChangePercent,marketCap,forwardPE,priceToBook,shortName,regularMarketPreviousClose,bid,ask,bidSize,askSize,messageBoardId,longName,financialCurrency,regularMarketChangePercent,regularMarketDayRange,averageDailyVolume3Month,averageDailyVolume10Day,fiftyTwoWeekLowChange,fiftyTwoWeekLowChangePercent,fiftyTwoWeekRange,fiftyTwoWeekHighChange,fiftyTwoWeekHighChangePercent,fiftyTwoWeekLow,fiftyTwoWeekHigh,ytdReturn,trailingThreeMonthReturns,trailingThreeMonthNavReturns,openInterest,underlyingSymbol,contractSymbol,expireIsoDate,expireDate,headSymbolAsString
count,81929,81929,81929,39875,7.705200e+04,77911,77911.000000,77911.000000,77911,77911,7.791100e+04,81929,81929,81929,80752.000000,77911,77911,77913,81929,66439,6.663300e+04,6.669600e+04,6.594900e+04,6.595800e+04,6.595200e+04,6.547100e+04,7.561000e+03,1.425000e+04,1.225600e+04,1.225600e+04,2.261000e+04,2.632800e+04,2.225800e+04,41118.000000,11173.000000,4.507600e+04,4.132600e+04,6.182200e+04,6.163400e+04,6.163400e+04,6.194900e+04,6.175100e+04,6.175100e+04,4.491800e+04,1.103400e+04,4.102700e+04,72063,6.603200e+04,6.419700e+04,6.422300e+04,6.077800e+04,6.078300e+04,44728,50279,46270,66603.0,65950,6.187200e+04,6.149000e+04,6.603100e+04,6.407300e+04,66247,6.603200e+04,6.476800e+04,6.624800e+04,6.624900e+04,158.000000,160.000000,110.000000,552.0,461,529,87,8.700000e+01,17
unique,1,1,11,3,NaN,4,NaN,NaN,39,23,NaN,1,2,2,NaN,68,43,70,81897,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55306,NaN,NaN,NaN,NaN,NaN,27472,32331,53,36666.0,43503,NaN,NaN,NaN,NaN,60390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,461,2,15,NaN,17
top,en-US,US,EQUITY,Delayed Quote,NaN,REGULAR,NaN,NaN,America/New_York,CEST,NaN,False,False,False,NaN,YHD,us_market,YHD,M,EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ICICI Prudential Mutual Fund,NaN,NaN,NaN,NaN,NaN,finmb_271199,BCE Inc.,USD,0.0,0.0 - 0.0,NaN,NaN,NaN,NaN,0.0 - 0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LICNFENGP.NS,True,2029-12-31T00:00:00Z,NaN,ZPLKRC-A
freq,81929,81929,67968,34634,NaN,58299,NaN,NaN,25066,27733,NaN,81929,68236,77292,NaN,11360,25059,11360,9,26976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36,NaN,NaN,NaN,NaN,NaN,26,24,12878,21110.0,1709,NaN,NaN,NaN,NaN,1174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,512,62,NaN,1
mean,NaN,NaN,NaN,NaN,1.546927e+09,NaN,1.720745,15.139775,NaN,NaN,3.370145e+06,NaN,NaN,NaN,2.674299,NaN,NaN,NaN,NaN,NaN,9.327983e+07,1.107619e+01,6.864768e+02,7.145243e+02,6.958736e+02,9.808521e+05,1.508375e+09,1.555817e+09,1.555508e+09,1.555753e+09,1.769164e+05,1.416268e+08,7.718715e+03,15.839277,12.370525,2.931069e+09,1.935805e+02,9.262514e+02,8.663150e+05,1.006714e+06,9.462080e+02,8.646521e+05,4.744239e+04,1.877742e+15,4.948297e+05,8.397206e+06,NaN,7.180272e+02,6.598850e+02,6.794441e+02,5.803993e+03,4.322104e+03,NaN,NaN,NaN,NaN,NaN,1.538937e+06,1.380417e+06,2.439880e+06,1.351984e+06,NaN,2.439392e+06,1.836278e+05,6.830483e+02,1.134904e+03,10.109747,1.128313,0.581818,0.0,NaN,NaN,NaN,1.799846e+09,NaN
std,NaN,NaN,NaN,NaN,1.675323e+08,NaN,5.232722,1.880897,NaN,NaN,1.585285e+07,NaN,NaN,NaN,0.930088,NaN,NaN,NaN,NaN,NaN,3.214351e+09,5.849570e+03,1.088795e+04,1.097693e+04,1

In [12]:
extras = pd.merge(stocks, infos, how='inner', left_on='ticker', right_on='symbol')

extras.head()

,ticker,name,exchange_x,category name,country,language,region,quoteType,quoteSourceName,regularMarketTime,marketState,exchangeDataDelayedBy,sourceInterval,exchangeTimezoneName,exchangeTimezoneShortName,gmtOffSetMilliseconds,esgPopulated,tradeable,triggerable,priceHint,fullExchangeName,market,exchange_y,symbol,currency,regularMarketPrice,regularMarketChange,regularMarketOpen,regularMarketDayHigh,regularMarketDayLow,regularMarketVolume,dividendDate,earningsTimestamp,earningsTimestampStart,earningsTimestampEnd,trailingAnnualDividendRate,trailingPE,trailingAnnualDividendYield,epsTrailingTwelveMonths,epsForward,sharesOutstanding,bookValue,fiftyDayAverage,fiftyDayAverageChange,fiftyDayAverageChangePercent,twoHundredDayAverage,twoHundredDayAverageChange,twoHundredDayAverageChangePercent,marketCap,forwardPE,priceToBook,shortName,regularMarketPreviousClose,bid,ask,bidSize,askSize,messageBoardId,longName,financialCurrency,regularMarketChangePercent,regularMarketDayRange,averageDailyVolume3Month,averageDailyVolume10Day,fiftyTwoWeekLowChange,fiftyTwoWeekLowChangePercent,fiftyTwoWeekRange,fiftyTwoWeekHighChange,fiftyTwoWeekHighChangePercent,fiftyTwoWeekLow,fiftyTwoWeekHigh,ytdReturn,trailingThreeMonthReturns,trailingThreeMonthNavReturns,openInterest,underlyingSymbol,contractSymbol,expireIsoDate,expireDate,headSymbolAsString
0,OEDV,"Osage Exploration and Development, Inc.",PNK,NaN,USA,en-US,US,MUTUALFUND,Delayed Quote,1.561760e+09,REGULAR,0.0,15.0,America/New_York,EDT,-14400000.0,False,False,False,2.0,YHD,us_market,YHD,OEDV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAPL,Apple Inc.,NMS,Electronic Equipment,USA,en-US,US,EQUITY,Nasdaq Real Time Price,1.566829e+09,REGULAR,0.0,15.0,America/New_York,EDT,-14400000.0,False,True,True,2.0,NasdaqGS,us_market,NMS,AAPL,USD,205.750,3.110001,205.86,207.1900,205.53,7381238.0,1.565827e+09,1.564531e+09,1.572448e+09,1.572887e+09,2.96,17.470493,0.014607,11.777,12.80,4.601080e+09,21.286,205.056560,0.693436,0.003382,193.434420,12.315582,0.063668,9.298212e+11,16.074219,9.665978,Apple Inc.,202.64,206.47,206.51,10.0,10.0,finmb_24937,Apple Inc.,USD,1.53474,205.53 - 207.19,26427811.0,28254233.0,63.750000,0.448944,142.0 - 233.47,-27.720001,-0.118730,142.00,233.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BAC,Bank of America Corporation,NYQ,Money Center Banks,USA,en-US,US,EQUITY,Nasdaq Real Time Price,1.566829e+09,REGULAR,0.0,15.0,America/New_York,EDT,-14400000.0,False,True,True,2.0,NYSE,us_market,NYQ,BAC,USD,26.525,0.055000,26.70,26.7400,26.50,7086157.0,1.569542e+09,1.571223e+09,1.515677e+09,1.516023e+09,0.60,9.439502,0.022667,2.810,3.04,9.508200e+09,26.408,28.741714,-2.216715,-0.077125,28.784782,-2.259783,-0.078506,2.469027e+11,8.725329,1.004430,Bank of America Corporation,26.47,26.59,26.60,361.0,14.0,finmb_19049,Bank of America Corporation,USD,0.207784,26.5 - 26.74,52258963.0,57081883.0,3.865000,0.170565,22.66 - 31.49,-4.965000,-0.157669,22.66,31.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AMZN,"Amazon.com, Inc.",NMS,Catalog & Mail Order Houses,USA,en-US,US,EQUITY,Nasdaq Real Time Price,1.566829e+09,REGULAR,0.0,15.0,America/New_York,EDT,-14400000.0,False,True,True,2.0,NasdaqGS,us_market,NMS,AMZN,USD,1756.000,6.380005,1766.91,1769.8206,1751.40,849731.0,NaN,1.564099e+09,1.571843e+09,1.572278e+09,NaN,72.850975,NaN,24.104,33.22,4.920530e+08,107.411,1888.932000,-132.932000,-0.070374,1824.243400,-68.243410,-0.037409,8.686159e+11,52.859722,16.348417,"Amazon.com, Inc.",1749.62,1757.77,1757.42,8.0,14.0,finmb_18749,"Amazon.com, Inc.",USD,0.364651,1751.4 - 1769.8206,3624549.0,2953316.0,449.000000,0.343535,1307.0 - 2050.5,-294.500000,-0.143623,1307.00,2050.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T,AT&T Inc.,NYQ,Telecom Services - Domestic,USA,en-US,US,EQUITY,Nasdaq Real Time Price,1.566829e+09,REGULAR,0.0,15.0,America/New_York,EDT,-14400000.0,False,True

In [13]:
extras.describe(include='all')

,ticker,name,exchange_x,category name,country,language,region,quoteType,quoteSourceName,regularMarketTime,marketState,exchangeDataDelayedBy,sourceInterval,exchangeTimezoneName,exchangeTimezoneShortName,gmtOffSetMilliseconds,esgPopulated,tradeable,triggerable,priceHint,fullExchangeName,market,exchange_y,symbol,currency,regularMarketPrice,regularMarketChange,regularMarketOpen,regularMarketDayHigh,regularMarketDayLow,regularMarketVolume,dividendDate,earningsTimestamp,earningsTimestampStart,earningsTimestampEnd,trailingAnnualDividendRate,trailingPE,trailingAnnualDividendYield,epsTrailingTwelveMonths,epsForward,sharesOutstanding,bookValue,fiftyDayAverage,fiftyDayAverageChange,fiftyDayAverageChangePercent,twoHundredDayAverage,twoHundredDayAverageChange,twoHundredDayAverageChangePercent,marketCap,forwardPE,priceToBook,shortName,regularMarketPreviousClose,bid,ask,bidSize,askSize,messageBoardId,longName,financialCurrency,regularMarketChangePercent,regularMarketDayRange,averageDailyVolume3Month,averageDailyVolume10Day,fiftyTwoWeekLowChange,fiftyTwoWeekLowChangePercent,fiftyTwoWeekRange,fiftyTwoWeekHighChange,fiftyTwoWeekHighChangePercent,fiftyTwoWeekLow,fiftyTwoWeekHigh,ytdReturn,trailingThreeMonthReturns,trailingThreeMonthNavReturns,openInterest,underlyingSymbol,contractSymbol,expireIsoDate,expireDate,headSymbolAsString
count,81920,80149,78938,20571,78938,81920,81920,81920,39866,7.704300e+04,77902,77902.000000,77902.000000,77902,77902,7.790200e+04,81920,81920,81920,80743.000000,77902,77902,77904,81920,66439,6.663300e+04,6.669600e+04,6.594900e+04,6.595800e+04,6.595200e+04,6.547100e+04,7.561000e+03,1.425000e+04,1.225600e+04,1.225600e+04,2.261000e+04,2.632800e+04,2.225800e+04,41118.000000,11173.000000,4.507600e+04,4.132600e+04,6.181300e+04,6.163400e+04,6.163400e+04,6.194000e+04,6.175100e+04,6.175100e+04,4.491800e+04,1.103400e+04,4.102700e+04,72059,6.603200e+04,6.419700e+04,6.422300e+04,6.077800e+04,6.078300e+04,44728,50279,46270,66603.0,65950,6.186300e+04,6.148100e+04,6.603100e+04,6.407300e+04,66238,6.603200e+04,6.476800e+04,6.623900e+04,6.624000e+04,158.000000,160.000000,110.000000,552.0,461,529,87,8.700000e+01,17
unique,81894,54557,64,214,41,1,1,11,3,NaN,4,NaN,NaN,39,23,NaN,1,2,2,NaN,68,43,70,81894,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55305,NaN,NaN,NaN,NaN,NaN,27472,32331,53,36666.0,43503,NaN,NaN,NaN,NaN,60388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,461,2,15,NaN,17
top,M,ICICI Prudential Mutual Fund,PNK,Industrial Metals & Minerals,USA,en-US,US,EQUITY,Delayed Quote,NaN,REGULAR,NaN,NaN,America/New_York,CEST,NaN,False,False,False,NaN,YHD,us_market,YHD,M,EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ICICI Prudential Mutual Fund,NaN,NaN,NaN,NaN,NaN,finmb_271199,BCE Inc.,USD,0.0,0.0 - 0.0,NaN,NaN,NaN,NaN,0.0 - 0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LICNFENGP.NS,True,2029-12-31T00:00:00Z,NaN,ZPLKRC-A
freq,9,34,11285,1281,17995,81920,81920,67968,34625,NaN,58290,NaN,NaN,25057,27733,NaN,81920,68227,77283,NaN,11351,25050,11351,9,26976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36,NaN,NaN,NaN,NaN,NaN,26,24,12878,21110.0,1709,NaN,NaN,NaN,NaN,1174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,512,62,NaN,1
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.546926e+09,NaN,1.720944,15.139791,NaN,NaN,3.372198e+06,NaN,NaN,NaN,2.674374,NaN,NaN,NaN,NaN,NaN,9.327983e+07,1.107619e+01,6.864768e+02,7.145243e+02,6.958736e+02,9.808521e+05,1.508375e+09,1.555817e+09,1.555508e+09,1.555753e+09,1.769164e+05,1.416268e+08,7.718715e+03,15.839277,12.370525,2.931069e+09,1.935805e+02,9.263863e+02,8.663150e+05,1.006714e+06,9.463455e+02,8.646521e+05,4.744239e+04,1.877742e+15,4.948297e+05,8.397206e+06,NaN,7.180272e+02,6.598850e+02,6.794441e+02,5.803993e+03,4.322104e+03,NaN,NaN,NaN,NaN,NaN,1.539157e+06,1.380615e+06,2.439880e+06,1.351984e+06,NaN,2.439392e+06,1.836278e+05,6.831411e+02,1.135058e+03,10.109747,1.128313,0.5

In [14]:
extras.to_csv('../datasets/extras.csv.gz')

In [15]:
def download_data(ticker, range='5y', interval='1d', events='div,splits'):
    url = f"{BASEURL}/v8/finance/chart/{ticker}"
    
    params = dict(
        range=range,
        events=events,
        interval=interval,
    )
    
    try:
        res = requests.get(url=url, params=params).json()
        result = res['chart']['result'][0]
        events = result['events']['dividends']
        indicators = result['indicators']['quote'][0]
                
        quotes = pd.DataFrame(indicators).assign(date=result['timestamp'], ticker=ticker)
        dividends = pd.DataFrame(events.values()).assign(ticker=ticker)
        
        return quotes, dividends
    except Exception as e:
        # fail silently
        return None

In [16]:
with ThreadPoolExecutor() as executor:
    gen = executor.map(download_data, tickers)
    values = filter(notnone, tqdm(gen, total=len(tickers)))
    quotes, dividends = zip(*values)

alldividends = pd.concat(dividends, sort=False)
allquotes = pd.concat(quotes, sort=False)

allquotes.shape, alldividends.shape

((43174418, 7), (352278, 3))

In [17]:
allquotes.head()

,close,volume,low,open,high,date,ticker
0,100.889999,33152000.0,100.860001,101.419998,101.500000,1409059800,AAPL
1,102.129997,52369000.0,100.699997,101.019997,102.570000,1409146200,AAPL
2,102.250000,68460000.0,101.559998,101.589996,102.779999,1409232600,AAPL
3,102.500000,44595000.0,102.199997,102.860001,102.900002,1409319000,AAPL
4,103.300003,53564000.0,102.720001,103.059998,103.739998,1409664600,AAPL


In [18]:
allquotes.describe(include='all')

,close,volume,low,open,high,date,ticker
count,4.270268e+07,4.270268e+07,4.270268e+07,4.270268e+07,4.270268e+07,4.317442e+07,43174418
unique,NaN,NaN,NaN,NaN,NaN,NaN,37669
top,NaN,NaN,NaN,NaN,NaN,NaN,GLYHO.IS
freq,NaN,NaN,NaN,NaN,NaN,NaN,1305
mean,1.156137e+03,2.198862e+06,1.163541e+03,1.182698e+03,1.199926e+03,1.489143e+09,NaN
std,1.772945e+05,1.643321e+08,4.182106e+04,4.321849e+04,4.408267e+04,4.541734e+07,NaN
min,-1.123859e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.409011e+09,NaN
25%,5.940000e+00,0.000000e+00,5.896000e+00,5.939000e+00,5.989000e+00,1.449817e+09,NaN
50%,2.021000e+01,1.000000e+02,2.007500e+01,2.020000e+01,2.035000e+01,1.489585e+09,NaN
75%,6.073000e+01,5.797800e+04,6.027000e+01,6.070000e+01,6.115000e+01,1.528679e+09,NaN


In [21]:
allquotes.to_csv('../datasets/quotes.csv.gz')

In [19]:
alldividends.head()

,amount,date,ticker
0,0.57,1478179800,AAPL
1,0.63,1494509400,AAPL
2,0.73,1541687400,AAPL
3,0.63,1510324200,AAPL
4,0.73,1526045400,AAPL


In [20]:
alldividends.describe(include='all')

,amount,date,ticker
count,352278.000000,3.522780e+05,352278
unique,NaN,NaN,37669
top,NaN,NaN,RELIGARE.BO
freq,NaN,NaN,87
mean,28.002358,1.490302e+09,NaN
std,2457.975662,4.573135e+07,NaN
min,0.000000,1.409015e+09,NaN
25%,0.090000,1.450681e+09,NaN
50%,0.250000,1.492582e+09,NaN
75%,0.700000,1.529388e+09,NaN


In [24]:
alldividends.to_csv('../datasets/dividends.csv.gz')